In [ ]:
from pathlib import Path
import pickle

import pandas
from pandas import DataFrame
import yaml


def log_as_dataframe(log):
    columns = ['timestamp', 'level', 'source', 'function', 'data']
    df = pandas.DataFrame(log, columns=columns)
    return df


df = log_as_dataframe(pickle.load(open('../log.pkl', 'rb')))
df = df[df['level'] == 'DATA']
df.head()

In [ ]:
data = []
for row in df.itertuples():
    try:
        value = yaml.load(row.data)
    except yaml.error.YAMLError:
        continue
    if len(value) != 12:
        print(value)
        continue
    data.append([row.timestamp] + value)
df = DataFrame(data)
df.columns = ['timestamp', 'front_left', 'front_right', 'side_left', 'side_right',
              'linear_ideal', 'linear_measured', 'angular_ideal', 'angular_measured',
              'voltage_left', 'voltage_right', 'pwm_left', 'pwm_right']
df['timestamp'] /= 1000
df = df.set_index('timestamp')
df = df.sort_index()
df.head()

In [ ]:
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure


output_notebook()

In [ ]:
def show_data(df):
    sensors = figure(width=800, height=400,
                     title='Front sensors log',
                     x_axis_label='Time (s)',
                     y_axis_label='Distance (m)',
                     y_range=(0., 0.4))
    sensors.line(x=df.index, y=df['front_left'], color='orange', legend='Front-left')
    sensors.line(x=df.index, y=df['front_right'], legend='Front-right')

    linear = figure(width=800, height=400,
                    title='Linear speed',
                    x_axis_label='Time (s)',
                    y_axis_label='Speed (m/s)',
                    x_range=sensors.x_range)
    linear.line(x=df.index, y=df['linear_ideal'], color='orange', legend='Ideal')
    linear.line(x=df.index, y=df['linear_measured'], legend='Measured')

    angular = figure(width=800, height=400,
                     title='Angular speed',
                     x_axis_label='Time (s)',
                     y_axis_label='Speed (rad/s)',
                     x_range=sensors.x_range)
    angular.line(x=df.index, y=df['angular_ideal'], color='orange', legend='Ideal')
    angular.line(x=df.index, y=df['angular_measured'], legend='Measured')

    pwm = figure(width=800, height=400,
                 title='Output PWM',
                 x_axis_label='Time (s)',
                 y_axis_label='Duty',
                 x_range=sensors.x_range)
    pwm.line(x=df.index, y=df['pwm_left'], color='orange', legend='Left')
    pwm.line(x=df.index, y=df['pwm_right'], legend='Right')

    grid = gridplot([sensors, linear, angular, pwm], ncols=1)

    show(grid)


show_data(df)